# Data

In [ ]:
 #!wget http://your_domain/klines_1m_BTC_USDT.parquet

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_parquet('klines_5m_BTC_USDT.parquet')
df

In [ ]:
df = df.resample('30min').apply({
    'open': 'first',
    'close': 'last',
    'low': 'min',
    'high': 'max',
})
df

In [ ]:
# Look at the change between open and close of a candle, normalized by the average price of btc at this time
data = 2 * (df['close'] - df['open']) / (df['close'] + df['open'])
sns.histplot(data, kde=True).set(xlim=(-0.02, 0.02))

In [ ]:
direction = (df['close'] - df['open'] > 0) * 1.0 - (df['close'] - df['open'] < 0) * 1.0
direction

In [ ]:
plt.scatter(direction.iloc[:250].index, direction.iloc[:250])
plt.show()

# Pytorch model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
